In [1]:
import os

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from neurostatx.utils.preprocessing import merge_dataframes


In [2]:
repository_path = "c:/Users/Rosalie/OneDrive - Office 365/Documents/UdeS/Hiver 2026/Crédits de recherche/GitHub/FuzzyClustering-PING/"
ping_base_path = "c:/Users/Rosalie/OneDrive - Office 365/Documents/UdeS/Hiver 2026/Crédits de recherche/data/PINGTabularData/"
ping_int_path = "c:/Users/Rosalie/OneDrive - Office 365/Documents/UdeS/Hiver 2026/Crédits de recherche/result/inversenormaltransformation/"
output_dir = "c:/Users/Rosalie/OneDrive - Office 365/Documents/UdeS/Hiver 2026/Crédits de recherche/result/datagathering/"
os.makedirs(output_dir, exist_ok=True)

Fetching relevent data from PING study

In [3]:
# Load all necessary dataframes for cognitive test
dccs_int = pd.read_excel(f"{ping_int_path}/dccs_int.xlsx")
flanker_int = pd.read_excel(f"{ping_int_path}/flanker_int.xlsx")
ibam_int = pd.read_excel(f"{ping_int_path}/ibam_int.xlsx")
lswmt_int = pd.read_excel(f"{ping_int_path}/lswmt_int.xlsx")
orrt_int = pd.read_excel(f"{ping_int_path}/orrt_int.xlsx")
pcps_int = pd.read_excel(f"{ping_int_path}/pcps_int.xlsx")
tpvt_int = pd.read_excel(f"{ping_int_path}/tpvt_int.xlsx")

# Load all necessary dataframes for covariates/demographics
# Age, Sex, Ethnicity, income, education, ADHD, handedness
demo = pd.read_csv(f"{ping_base_path}/fdh01.txt", sep="\t")


C:\Users\Rosalie\AppData\Local\Temp\ipykernel_11260\2955698399.py:12: DtypeWarning: Columns (0,1,2,6,10,11,13,14,16,17,18,19,20,21,22,24,25,26,27,28,29,30,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,55,56,57,58,59,60,68,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,2

In [ ]:
demo_base = demo.iloc[1:].copy()

# Extracting age data
age = demo_base[["subjectkey", "interview_age"]]
age.columns = ["subjectkey", "age_months"]
age["age_months"] = pd.to_numeric(demo_base["interview_age"], errors="coerce")

# Extracting demographics data
demo_vars = [
    "subjectkey",
    "sex",
    "behavobs_handedness",
    "race_w",
    "race_b", 
    "ethnicity",
    "race_othera",
    "race_ia",
    "race_otherpi",
    "fdh_grdian_1_edu",
    "fdh_grdian_2_edu",
    "fdh_3_hhold_income"
]

demo_data = demo_base[demo_vars].copy()
demo_data.columns = [
    "subjectkey",
    "sex",
    "handedness",
    "eth_white",
    "eth_black",
    "eth_hispanic_latino",
    "eth_asia",
    "eth_indian_ame",
    "eth_pacific_islander",
    "parent_ed1",
    "parent_ed2",
    "income"
]

#  Transform the handedness score.  1 = left, 2 = right, 3 = ambidex
def transform_handedness(x):
    if x == "L":
        return 1
    elif x == "R":
        return 2
    elif x == "A":
        return 3
    else :
        return np.nan

demo_data.loc[:, "handedness"] = demo_data["handedness"].apply(transform_handedness)

# Transform sex data. 1 = male, 2 = female, 0 = unknown
def transform_sex(x):
    if x == "M":
        return 1
    elif x == "F":
        return 2
    else:
        return 0

demo_data.loc[:, "sex"] = demo_data["sex"].apply(transform_sex)

# Extract parental highest education level. Taking the highest among the two parents.
for col in ["parent_ed1", "parent_ed2"]:
    demo_data[col] = (
        demo_data[col]
        .replace([777, 999, "Missing value", "Other", np.nan], 0)
    )
    demo_data[col] = pd.to_numeric(demo_data[col], errors="coerce")
demo_data.loc[:, "high_edu"] = demo_data[["parent_ed1", "parent_ed2"]].fillna(0).max(axis=1)

# Create high parent education groups. 1 = no high school, 2 = high school, 3 = some college, 4 = bachelor, 5 = post-graduate, 0 = unknown
def create_edu_groups(x):
    if x < 4:
        return 1 # no high school
    elif x == 4:
        return 2 # high school
    elif x == 5:
        return 3 # some college
    elif x == 6: 
        return 4 # bachelor
    elif x == 7 :
        return 5  # post-graduate
    else:
        return 0

demo_data.loc[:, "edu_groups"] = demo_data["high_edu"].apply(create_edu_groups)

# Regrouping ethnicity
cols = [
    "eth_white",
    "eth_black",
    "eth_asia",
    "eth_indian_ame",
    "eth_pacific_islander"
]

# Convert to numeric
demo_data[cols] = demo_data[cols].apply(pd.to_numeric, errors="coerce")

# Recode 1 = yes, 0 = no
demo_data[cols] = (demo_data[cols] == 1).astype(int)

# Create a new column with the combined ethnicity categories. 1 = White, 2 = Black, 3 = Hispanic/Latino, 4 = Asian, 5 = Multiethnic, 6 = Other, 0 = Unknown
def combine_ethnicity(row):
    race_flags = [
        row["eth_white"],
        row["eth_black"],
        row["eth_asia"],
        row["eth_indian_ame"],
        row["eth_pacific_islander"]
    ]
    race_count = sum(race_flags)
    is_hispanic = str(row["eth_hispanic_latino"]).strip().lower() == "hispanic or latino"

    # Multiethnic 
    if is_hispanic and race_count >= 1:
        return 5  # Multiethnic
    if race_count > 1:
        return 5  # Multiethnic

    # Hispanic only
    if is_hispanic:
        return 3

    # Etnicity categories
    if row["eth_white"] == 1:
        return 1 # White
    elif row["eth_black"] == 1:
        return 2 # Black
    elif row["eth_asia"] == 1:
        return 4 # Asian
    elif row["eth_indian_ame"] == 1 or row["eth_pacific_islander"] == 1:
        return 6  # Other
    else:
        return 0  # Unknown

demo_data["combine_ethnicity"] = demo_data.apply(combine_ethnicity, axis=1)

# Create new income groups into 1 = <50 000, 2 = 50 000 - 99 999, 3 = 100 000 +, 0 = unknown
demo_data["income"] = (demo_data["income"].replace(["Missing value", 777, 999, np.nan], 0))
demo_data["income"] = pd.to_numeric(demo_data["income"], errors="coerce")

def create_income_groups(x):
    if pd.isna(x):
        return 0
    if x < 7:
        return 1 # <50 000
    elif x == 7 :
        return 2 # 50 000 - 99 999
    elif x in [8, 9, 10, 11, 12]:
        return 3 # 100 000 +
    else:
        return 0

demo_data.loc[:, "income_groups"] = demo_data["income"].apply(create_income_groups)



C:\Users\Rosalie\AppData\Local\Temp\ipykernel_11260\2887618889.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  age["age_months"] = pd.to_numeric(demo_base["interview_age"], errors="coerce")


In [ ]:
# Copy baseline data for each cognitive test
dccs_base = dccs_int.iloc[1:].copy()
flanker_base = flanker_int.iloc[1:].copy()
ibam_base = ibam_int.iloc[1:].copy()
lswmt_base = lswmt_int.iloc[1:].copy()
orrt_base = orrt_int.iloc[1:].copy()
pcps_base = pcps_int.iloc[1:].copy()
tpvt_base = tpvt_int.iloc[1:].copy()

# Create a function to scale the scores for each test
def scale_test(df, original_col, new_col):
    df = df[["subjectkey", original_col]].copy()
    df.columns = ["subjectkey", new_col]
    df.loc[:, new_col] = StandardScaler().fit_transform(df[[new_col]])
    return df

# Scale scores for each test using the function
dccs_base    = scale_test(dccs_base,    "cardsort_scaled_score",    "card_sort")
flanker_base = scale_test(flanker_base, "flanker_scaled_score",     "flanker")
ibam_base    = scale_test(ibam_base,    "ibam_scaled_score",        "imitation_memory")
lswmt_base   = scale_test(lswmt_base,   "listsorting_scaled_score", "list_sorting")
orrt_base    = scale_test(orrt_base,    "oralreading_scaled_score", "oral_reading")
pcps_base    = scale_test(pcps_base,    "patterncomp_scaled_score", "pattern_comparison")
tpvt_base    = scale_test(tpvt_base,    "pictvocab_scaled_score",   "picture_vocab")

# Import ADHD diagnosis data
adhd_base = demo_base[["subjectkey", "cast39_adhd"]]
adhd_base.columns = ["subjectkey", "ADHD"]
adhd_base.loc[:, "ADHD"] = adhd_base["ADHD"].replace(["Missing value", 777, 999], np.nan)
adhd_base.loc[:, "ADHD"] = pd.to_numeric(adhd_base["ADHD"], errors="coerce")

In [ ]:
# Merge cognitive data fist so this way, we avoid the loss of sujects due to missing data in demographics columns
psy_behav = merge_dataframes({"age_months" : age, "ADHD" : adhd_base, "card_sort": dccs_base, "flanker" : flanker_base,
                               "imitation_memory" : ibam_base, "list_sorting" : lswmt_base, "oral_reading" : orrt_base,
                                 "pattern_comparison" : pcps_base, "picture_vocab" : tpvt_base}, index="subjectkey")

# Remove subjects with missing data in any of the columns
psy_behav.dropna(inplace=True, axis=0)
psy_behav.reset_index(drop=False, inplace=True)
print("Number of subjects retained for the analysis: {}".format(psy_behav.shape[0]))

Number of subjects retained for the analysis: 1238


In [ ]:
# Remove unnecessary individuals columns from the demographics dataframe and keep only the combined ethnicity column
demo_filtered = demo_data[
    ["subjectkey", "sex", "combine_ethnicity",
     "parent_ed1", "parent_ed2", "high_edu", "edu_groups",
     "income", "income_groups", "handedness"]
].copy()

# Concatenating all the dataframes
ping_data_gathered = merge_dataframes({"demo" : demo_filtered, "psy_behav" : psy_behav}, index="subjectkey")

# Droping row with NAN in the last 8 colums wich corresponds to the cognitive test scores and ADHD diagnosis
ping_data_gathered.dropna(inplace=True, axis=0, subset=ping_data_gathered.columns[-8:], how="all")

# Droping 10 subjects with missing data in the handedness column
ping_data_gathered.dropna(subset=["handedness"], inplace=True)

# Reordering the columns
ping_data_gathered = ping_data_gathered[["sex", "age_months", "combine_ethnicity",
                                          "parent_ed1", "parent_ed2", "high_edu", "edu_groups",
                                          "income", "income_groups", "handedness", "ADHD",
                                          "card_sort", "flanker", "imitation_memory", "list_sorting",
                                          "oral_reading", "pattern_comparison", "picture_vocab"]]

# Renaming the columns
ping_data_gathered.columns = ["Sex", "AgeMonths", "Ethnicity",
                                "Parent_ed1", "Parent_ed2", "high_edu", "edu_groups",
                                "Income", "income_groups", "Handedness", "ADHD",
                                "CardSort", "Flanker", "ImitationMemory", "ListSorting",
                                "OralReading", "PatternComparison", "PictureVocab"]

# Assert the number of subjects is the same as before -10 subjects with missing data in the handedness column
assert ping_data_gathered.shape[0] == psy_behav.shape[0] - 10, "Number of subjects do not match."

# Save the final dataframe
ping_data_gathered.to_excel(f"{output_dir}/ping_data_gathered.xlsx", index=True, header=True)